# MNL

In [105]:
import gurobipy as gp

In [3]:
import numpy as np
import numpy.random as npr
N = 25 #25 50 75 100
M = 1 #1 5 10 20
v = 100*npr.rand(M, N)
alpha = 1/M * np.ones(M)
print(v)

[[13.35170255 46.54432254 56.86309786 21.43202943 73.25531198 89.61114077
  80.45322793 97.57676508 38.72627304 67.54168511 88.61789608  5.93492513
  74.41367756 95.32468076 17.32079351 47.01134686 18.05619512 91.17590391
  37.8239436  34.95089283 39.51128514 67.05584336 44.62500172 44.5396566
  96.94684023]
 [22.465747   53.15715345 94.02010377 18.74348167  1.97805115 27.65164466
  38.46822225 55.14946082 94.81690982 34.47206892 32.25299884 63.90109366
  67.31036667 95.42802791 69.91624882 54.57853732 64.63762812 97.8988863
  34.16496855 94.42493106 52.03287149 61.97525742 45.90526387 49.95355367
  32.31934383]
 [63.91802611 66.81594589 59.06751052 95.15870299 63.72479788 28.03990218
  75.80984036 17.08077104 75.2957967  14.21096738 52.2339305  11.50023091
  78.59377966 49.610435   75.79626528 66.57691743  9.8855433  51.61604771
  88.99052397 63.01238607  1.25931594 96.5984755  50.45638244 16.32675665
  30.39300627]
 [95.58900548 24.50573101 22.4891079  49.00396766 92.98559169 72.3654

In [1]:
def IP_mmnl(p):
    global v, N
    global theta, M

    #create a new model
    myModel = gp.Model("IP_mmnl")

    # create decision variables and store them in the arrays z, x, y
    z = [0 for j in range(M)]
    x = [[0 for i in range(N)] for j in range(M)]
    y = [0 for i in range(N)]
    for j in range(M):
        newVar = myModel.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = "z_" + str(j))
        z[j] = newVar
        for i in range(N):
            newVarx = myModel.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = "x_" + str(i) + '_' + str(j))
            x[i][j] = newVarx
    for i in range(N):
        newVar = myModel.addVar(vtype = gp.GRB.BINARY, name = "y_" + str(i))
        y[i] = newVar
    myModel.update()

    # set objective function
    objExpr = gp.LinExpr()
    for j in range(M):
        objExpr += theta[j] * z[j]
    myModel.setObjective(objExpr, gp.GRB.MAXIMIZE)

    B = 100*max(p)
    # add constraints
    for j in range(M):
        myConstr = gp.LinExpr()
        for i in range(N):
            myConstr += v[i][j] * x[i][j]
            myModel.addConstr(lhs = x[i][j], sense = gp.GRB.LESS_EQUAL, rhs = y[i]*B, name = "x 1st constraint_" + str(i) + '_' + str(j))
            myModel.addConstr(lhs = x[i][j], sense = gp.GRB.GREATER_EQUAL, rhs = v[i][j] - z[j] - (1-y[i])*B, name = "x 2nd constraint_" + str(i) + '_' + str(j))
            myModel.addConstr(lhs = x[i][j], sense = gp.GRB.LESS_EQUAL, rhs = v[i][j] - z[j] + (1-y[i])*B, name = "x 3rd constraint_" + str(i) + '_' + str(j))
        myModel.addConstr(lhs = z[j], sense = gp.GRB.LESS_EQUAL, rhs = myConstr, name = "z " + str(j))

    # solve model
    myModel.optimize()

    bestS = []
    profit = float(myModel.objVal)
    for i in range(N):
        if y[i].x == 1:
            bestS.append(i)
    bestS = sorted(bestS)
    return profit, bestS

In [108]:
def profit_mmnl(p, S):
    global v
    global theta, M
    pi = 0
    for j in range(M):
        Vj_S = 0
        num = 0
        for i in S:
            num += p[i]*v[i][j]
            Vj_S += v[i][j]
        pi += theta[j]*num/(1+Vj_S)
    return pi

In [109]:
def nested_by_price_mmnl(p):
    global N
    profits = [0 for i in range(N+1)]
    S = []
    for i in range(N):
        S.append(i)
        profits[i+1] = profit_mmnl(p, S)
    best_S = [i for i in range(np.argmax(profits))]
    return max(profits), best_S

In [110]:
def find_best_option_mmnl(p, S):
    global v, N
    current_best = profit_mmnl(p, S)
    bestS = S
    flag = 0
    N_S = [k for k in range(N) if k not in S]
    for k in N_S:
        newS = S + [k]
        if profit_mmnl(p, newS) >= current_best:
            flag = 1
            bestS = newS
            current_best = profit_mmnl(p, newS)
    return current_best, bestS, flag

def greedy_mmnl(p):
    global v, N
    max_profit = -1
    best_S = []
    S = []
    flag = 1
    while flag==1:
        max_profit, S, flag = find_best_option_mmnl(p, S)
    best_S = sorted(S)
    return max_profit, best_S

In [111]:
import matplotlib.pyplot as plt
import math
import statistics as st
import time

In [112]:
n_simulations = 10000
avg_price = 100
std_price = 50

In [113]:
count_g = 0
result_g = np.zeros(n_simulations)
for i in range(n_simulations):
    p = npr.exponential(avg_price, N)
    # p = npr.normal(avg_price, std_price, N)
    p = sorted(p, reverse = True)
    gr = greedy_mmnl(p)
    mipl = IP_mmnl(p)
    result_g[i] = 100*gr[0]/mipl[0]
    count_g+=1
    if math.isclose(gr[0], mipl[0], rel_tol=1e-3):
        count_g-=1
print(count_g)
print(min(result_g))
print(st.mean(result_g))
print(max(result_g))

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 130 rows, 130 columns and 380 nonzeros
Model fingerprint: 0x2b4bb09f
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e-01, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+02]
Presolve time: 0.00s
Presolved: 130 rows, 130 columns, 380 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   8.969701e+03   0.000000e+00      0s
      24    2.8963698e+02   0.000000e+00   0.000000e+00      0s

Solved in 24 iterations and 0.01 seconds
Optimal objective  2.896369776e+02


IndexError: list index out of range